# Tema

## Data

In [ ]:
import newspaper
import pandas as pd
from tqdm import tqdm

data = pd.read_csv("data/contenido_2023_04_05_matriz_not_julia.csv")
data

In [ ]:
# Eliminar filas donde "contenido" esté vacío o NaN
data = data.dropna(subset=['contenido'])

# Verificar que ya no haya valores NaN
print(data['contenido'].isna().sum())


## Labels

In [27]:
label2id = {
    "Otros": 0,
    "Economía": 1, 
    "Expertos/as en IA": 2, 
    "Política": 3, 
    "Empresa": 4, 
    "Robótica": 5,
    "Aplicaciones / software": 6, 
    "Comunicación": 7, 
    "Investigación": 8, 
    "Algoritmo": 9,
    "Sesgos de género": 10, 
    "Sesgos de género y raza": 11, 
    "Premio": 12, 
    "Educación/cultura": 13,
}

id2label = {
    0: "Otros",
    1: "Economía", 
    2: "Expertos/as en IA", 
    3: "Política", 
    4: "Empresa", 
    5: "Robótica",
    6: "Aplicaciones / software", 
    7: "Comunicación", 
    8: "Investigación", 
    9: "Algoritmo",
    10: "Sesgos de género", 
    11: "Sesgos de género y raza", 
    12: "Premio",
    13: "Educación/cultura",
}



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Contar la cantidad de artículos por tema
tema_counts = data['Tema'].value_counts()

# Crear el gráfico
plt.figure(figsize=(12, 6))
ax = sns.barplot(x=tema_counts.index-1, y=tema_counts.values, palette="viridis")

# Agregar los números en cada barra con la posición correcta
for p in ax.patches:
    ax.annotate(f"{int(p.get_height())}", 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', 
                fontsize=12, fontweight='bold', color='black')

# Rotar las etiquetas del eje x
plt.xticks(rotation=45, ha="right")

# Agregar títulos y etiquetas
plt.xlabel("Tema")
plt.ylabel("Número de artículos")
plt.title("Distribución de artículos por tema")

# Mostrar el gráfico
plt.show()


### Balanceo de clases

In [ ]:
from sklearn.utils import resample

# Obtener la cantidad máxima de artículos en una clase
max_count = data['Tema'].value_counts().max()

# Lista para almacenar los nuevos datos balanceados
balanced_data = []

# Aplicar sobremuestreo a cada clase
for tema, subset in data.groupby('Tema'):
    balanced_subset = resample(subset, replace=True, n_samples=max_count, random_state=42)
    balanced_data.append(balanced_subset)

# Combinar todas las clases balanceadas
balanced_data = pd.concat(balanced_data)
data = balanced_data

# Mostrar la distribución después del balanceo
print(balanced_data['Tema'].value_counts())

# Graficar la nueva distribución
plt.figure(figsize=(12, 6))
ax = sns.barplot(x=balanced_data['Tema'].value_counts().index, 
                 y=balanced_data['Tema'].value_counts().values, palette="viridis")

# Agregar números en cada barra
for p in ax.patches:
    ax.annotate(f"{int(p.get_height())}", 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=12, fontweight='bold', color='black')

plt.xticks(rotation=45, ha="right")
plt.xlabel("Tema")
plt.ylabel("Número de artículos")
plt.title("Distribución de artículos después del balanceo")
plt.show()


## Train

In [30]:
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    data['contenido_clean'], data['Tema'], test_size=0.2, random_state=42, stratify=data['Tema'],
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Crear pipeline de vectorización y modelo
model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

# Entrenar modelo
model.fit(X_train, y_train)

# Evaluar modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=id2label.values()))


## Explicabilidad

### Por artículo

In [ ]:
import numpy as np
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=list(id2label.values()))

def explain_instance_predicted_class(text, num_words=20):
    # Obtener la clase predicha por el modelo
    predicted_class = np.argmax(model.predict_proba([text]))

    exp = explainer.explain_instance(
        text, 
        model.predict_proba, 
        num_features=num_words,  # Número de palabras a mostrar
        labels=[predicted_class]  # Explicar solo la clase predicha
    )

    print(f"Clase predicha: {id2label[predicted_class]}")  # Mostrar nombre de la clase
    return exp.show_in_notebook()

# Explicar un texto con su clase predicha
explain_instance_predicted_class(X_test.iloc[3])


### Por clase

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def explain_instance_all_classes_heatmap(text, num_words=5):
    # Obtener probabilidades del modelo
    proba = model.predict_proba([text])[0]
    predicted_class = np.argmax(proba)  # Clase predicha

    # Generar la explicación con LIME
    exp = explainer.explain_instance(
        text, 
        model.predict_proba, 
        num_features=num_words,
        labels=list(range(len(id2label)))  # Explicar TODAS las clases
    )

    print(f"Clase predicha: {id2label[predicted_class]}")
    print(f"Probabilidades de cada clase:")
    for i, prob in enumerate(proba):
        print(f"  {id2label[i]}: {prob:.2%}")

    # Crear una matriz donde cada fila es una palabra y cada columna una clase
    word_importance = {}

    for label in range(len(id2label)):
        exp_list = exp.as_list(label=label)  # Explicación de la clase actual
        for word, weight in exp_list:
            if word not in word_importance:
                word_importance[word] = [0] * len(id2label)  # Inicializar todas las clases en 0
            word_importance[word][label] = weight  # Asignar peso

    # Convertir a matriz para seaborn
    words = list(word_importance.keys())
    importance_matrix = np.array(list(word_importance.values()))

    # Crear el heatmap
    plt.figure(figsize=(12, len(words) * 0.5))
    sns.heatmap(
        importance_matrix, 
        xticklabels=[id2label[i] for i in range(len(id2label))], 
        yticklabels=words, 
        cmap="coolwarm", 
        annot=True, 
        fmt=".2f"
    )
    plt.xlabel("Clases")
    plt.ylabel("Palabras")
    plt.title(f"Importancia de palabras en todas las clases - Texto seleccionado")
    plt.show()

# Explicar un texto con Heatmap para todas las clases
explain_instance_all_classes_heatmap(X_test.iloc[3])


### Con mayor confianza

In [ ]:
# Obtener las predicciones de todo X_test
probs = model.predict_proba(X_test)

# Obtener la confianza más alta por cada texto
confidence_scores = np.max(probs, axis=1)

# Ordenar los textos por confianza descendente
top_indices = np.argsort(confidence_scores)[::-1][:5]  # Top 5 textos mejor clasificados

# Explicar los mejores clasificados
for i in top_indices:
    print(f"\nExplicando texto {i} con confianza {confidence_scores[i]:.2f}")
    explain_instance_predicted_class(X_test.iloc[i])
